In [59]:
#Imports 
import sys
from py2neo import neo4j, Graph
import MySQLdb


In [74]:
# Create MySQL connection
user = "lcdebaca"
password = "DcMCyskH49CKFnvEjx5C"
database = "starmetrics"

# invoke the connect() function, passing parameters in variables.
db = MySQLdb.connect( user = user, passwd = password, db = database )

# output basic database connection info.
print( db )

cursor = db.cursor( MySQLdb.cursors.DictCursor )
# Print Curser
print(cursor)

<_mysql.connection open to 'localhost' at 2b3b7c0>


In [75]:
# Test SQL 


# The Following is a Query of the starmetrics database
# It collects the following variables for Purdue University between the 
#years 2008 - 2012:  
#From the employee Table: id, employeeid, uniqueawardnumber, employeeoccupation
#From the occupation Table: occupationalclassification 

sql_select = "SElECT employee.employeeid, employee.uniqueawardnumber, employee.occupation_orig, \
occupation.occupationalclassification \
FROM starmetrics.employee \
INNER JOIN starmetrics.occupation ON \
starmetrics.employee.occupation_orig = occupation.occupation_orig \
WHERE starmetrics.employee.university = 'purdue' AND starmetrics.employee.year BETWEEN 2008 AND 2012;"

result_count = cursor.execute(sql_select);

# Count the number of rows returned 
print("Found " + str(result_count) + " rows")

# Testing
one = cursor.fetchone()
print(one.keys())
print(one)


# Store the results of the Query in a list
results = cursor.fetchall()

Found 1027382 rows
['occupationalclassification', 'occupation_orig', 'employeeid', 'uniqueawardnumber']
{'occupationalclassification': 'Faculty', 'occupation_orig': 'FACULTY', 'employeeid': '90014540', 'uniqueawardnumber': '00.070 03-C-NE-PU'}


I'm really not sure what is happening on the server when i run this stuff, so i'm going to try and be a bit conservative 

In [64]:
# Create connection to the Neo4jsdatabase using defaults
# note, I imported Graph when I imported py2neo
graph_db_test = Graph()



In [67]:
# First Things First,
# Below is an exaple of how to create a single node with the properties employeeid, uniqueidnumber, occupation_orig, 
#and occupation identification. 
# If we wanted to make a single node, this would be the syntax 
new_node = neo4j.Node("employee", employeeid = '90014540', uniqueawardnumber = '00.070 03-C-NE-PU', \
            occupation_orig = "FACULTY", occupationalclassification = "Faculty")

# Python uses a slightly different syntax to query the Neo4js database
#If we were using the Neo4js Turminal, this would be the Cypher Query we would type:
## CREATE (Employee1:Employee { id_num: "4", employeeid:'90014540', uniqueawardnumber:'00.070 03-C-NE-PU',occupation_orig:"FACULTY", occupationalclassification:"Faculty" })



In [ ]:
# If you want to test Cypher query syntax, This is a good site to experiemtn with;
#Cypher playground: http://console.neo4j.org/?_ga=1.113286210.459216022.1444237641

In [76]:
# Above I saved the results of the SQL Query into a list called results
# Below I iterate over that list, create properties based on the varibles in the SQL database,
# and finaly create an 'employee' node with those properties

for employee in results:
    id_num = employee['employeeid']
    occ_orig = employee['occupation_orig']
    occ_class = employee['occupationalclassification']
    award_num = employee['uniqueawardnumber']
    
    node = neo4j.Node("employee", employeeid = id_num, uniqueawardnumber = award_num, \
            occupation_orig = occ_orig, occupationalclassification = occ_class)
    


Now every employee is a node. If you could vizualize the graph now, it would look like a bunch of dots. There is not yet any relationships between the nodes to tie them together.  

In [80]:
# Jsut to prove that these employees were moved into the NEo databse, 
#lets query the database and collect 10 nodes
some_employees = graph_db_test.cypher.execute("MATCH (n) RETURN n.name LIMIT 10")

print some_employees 

    | n.name
----+--------
  1 | Phil  
  2 | Liz   
  3 | Chaz  
  4 | Anne  
  5 | Andy  
  6 | Ed    
  7 | Phil  
  8 | Liz   
  9 | Chaz  
 10 | Anne  



In [13]:
# Close Cursor
cursor.close()

# close connection
db.close()